In [1]:
library(data.table)
library(stringr)
library(dplyr)
library(ggplot2)
library(readxl)


Attaching package: 'dplyr'


The following objects are masked from 'package:data.table':

    between, first, last


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [2]:
myfiles24 <- Sys.glob("/anvil/scratch/x-walte247/tempfolder/*2024 WORKBOOK.csv")
fixfilename2024 <- function(x) str_to_title(str_remove(str_remove(x, "/anvil/scratch/x-walte247/tempfolder/"),  " 2024 WORKBOOK[A-Za-z \\.]*"))
names(myfiles24) <- sapply(myfiles24, fixfilename2024)
options(repr.matrix.max.cols=100)

In [3]:
makecountyDF <- function(x) {
  tempDF <- cbind(names(myfiles24)[x], fread(myfiles24[x]))
  names(tempDF)[1] <- "County"
  return(tempDF)
}

In [4]:
myresults24 <- lapply(1:length(myfiles24), makecountyDF)
names(myresults24) <- names(myfiles24)
sapply(myresults24, dim)

Adams,Allen,Bartholomew,Benton,Blackford,Boone,Brown,Carroll,Cass,Clark,Clay,Clinton,Crawford,Daviess,Dearborn,Decatur,Dekalb,Delaware,Dubois,Elkhart,Fayette,Fountain,Franklin,Fulton,Gibson,Grant,Greene,Huntington,Jackson,Jasper,Jay,Jefferson,Jennings,Johnson,Knox,Kosciusko,Lagrange,Laporte,Marshall,Miami,Monroe,Montgomery,Morgan,Newton,Noble,Ohio,Perry,Pike,Porter,Posey,Pulaski,Putnam,Randolph,Ripley,Rush,Scott,Shelby,Spencer,St. Joseph,Starke,Steuben,Sullivan,Switzerland,Tipton,Union,Vanderburgh,Vermillion,Vigo,Wabash,Warrick,Washington,Wayne,Wells,White,Whitley
19632,165863,38466,7803,8742,37535,15738,21380,25224,62209,23762,23154,11203,22288,32294,17782,31403,59291,35166,87544,14094,14512,17652,17216,22085,36849,26670,22760,26586,21404,15639,20252,20555,67433,31050,52177,27565,64656,30306,22757,55314,23453,42091,11338,26837,3520,16415,20182,83338,19693,17941,25828,18464,17849,12049,15294,30102,22066,118678,19773,39195,19415,8357,12152,6433,79307,12620,59828,19432,43006,19485,35109,16412,21634,20335
20,20,18,20,20,19,18,20,20,18,18,20,21,20,19,20,20,20,18,20,20,17,17,20,19,20,17,19,19,17,20,19,19,19,19,20,20,20,20,20,20,20,20,20,20,20,18,20,23,19,18,20,20,20,17,19,20,18,20,17,20,17,20,20,20,20,17,19,20,19,19,17,20,20,18


In [5]:
# Creating 2024 Statewide Workbook
# get parcel numbers
myparcelnumbers <- lapply(1:75, function(x) myresults24[[x]]$"Parcel Number")
which(sapply(myparcelnumbers, is.null))
##myparcelnumbers[[33]] <- myresults24[[33]]$"State Parcel Number"
table(sapply(myparcelnumbers, class))
# get prior year property class
myprioryearpropertyclass <- lapply(1:75, function(x) myresults24[[x]]$"Prior Year Property Class")
which(sapply(myprioryearpropertyclass, is.null))
##myprioryearpropertyclass[[7]] <- myresults[[7]]$"Prior  Year Property Class"
##myprioryearpropertyclass[[10]] <- myresults[[10]]$"Prior  Year Property Class"
##myprioryearpropertyclass[[33]] <- rep(NA, nrow(myresults[[33]]))
table(sapply(myprioryearpropertyclass, class))
# get current year property class
mycurrentyearpropertyclass <- lapply(1:75, function(x) myresults24[[x]]$"Current Year Property Class")
which(sapply(mycurrentyearpropertyclass, is.null))
##mycurrentyearpropertyclass[[33]] <- myresults[[33]]$"Property Class"
table(sapply(mycurrentyearpropertyclass, class))
myprioryeartotalAV <- lapply(1:75, function(x) as.numeric(gsub(",", "", myresults24[[x]]$"Prior Year Total AV")))
which(sapply(mycurrentyearpropertyclass, is.null))
myresults24[[11]]$"Prior Year Total AV" <- gsub("\\$", "", myresults24[[11]]$"Prior Year Total AV")
myprioryeartotalAV[[11]] <- as.numeric(gsub(",", "", myresults24[[11]]$"Prior Year Total AV"))
which(is.na(myprioryeartotalAV))
table(sapply(myprioryeartotalAV, class))
# get current year total AV
mycurrentyeartotalAV <- lapply(1:75, function(x) as.numeric(gsub(",", "", myresults24[[x]]$"Current Year Total AV")))
which(sapply(mycurrentyeartotalAV, is.null))
myresults24[[11]]$"Current Year Total AV" <- gsub("\\$", "", myresults24[[11]]$"Current Year Total AV")
mycurrentyeartotalAV[[11]] <- as.numeric(gsub(",", "", myresults24[[11]]$"Current Year Total AV"))
which(is.na(myprioryeartotalAV))
##mycurrentyeartotalAV[[33]] <- as.numeric(gsub("\\$", "", gsub(",", "", myresults[[33]]$"WIP Total AV")))
table(sapply(mycurrentyeartotalAV, class))

integer(0)


character 
       75 

integer(0)


integer 
     75 

integer(0)

...

Warning message in FUN(X[[i]], ...):
"NAs introduced by coercion"


integer(0)

Warning message:
"NAs introduced by coercion"


integer(0)


numeric 
     75 

In [6]:
# 2024 Statewide Workbook
combinedwkbk24 = data.frame(
  County = character(),
  ParcelNumber = character(),
  PriorPropertyClass = integer(),
  CurrentPropertyClass = integer(), 
  PriorAV = numeric(),
  CurrentAV = numeric()
)
for (i in 1:75) {
    newdata = data.frame(
      County = myresults24[[i]]$'County',
      ParcelNumber = myparcelnumbers[[i]],
      PriorPropertyClass = myprioryearpropertyclass[[i]],
      CurrentPropertyClass = mycurrentyearpropertyclass[[i]], 
      PriorAV = myprioryeartotalAV[[i]],
      CurrentAV = mycurrentyeartotalAV[[i]]
    )
    combinedwkbk24 = rbind(combinedwkbk24,newdata)
}
head(combinedwkbk24)

,County,ParcelNumber,PriorPropertyClass,CurrentPropertyClass,PriorAV,CurrentAV
,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>
1,Adams,01-01-01-100-001.000-012,101,101,231500,250000
2,Adams,01-01-01-100-002.000-012,100,100,69700,83700
3,Adams,01-01-01-100-002.500-012,100,100,92000,110400
4,Adams,01-01-01-100-004.000-012,685,685,244100,260400
5,Adams,01-01-01-100-005.000-012,610,610,0,0
6,Adams,01-01-01-100-006.000-012,610,610,0,0


In [7]:
myfiles23 <- Sys.glob("/anvil/scratch/x-walte247/tempfolder/*2023 WORKBOOK.csv")
fixfilename2023 <- function(x) str_to_title(str_remove(str_remove(x, "/anvil/scratch/x-walte247/tempfolder/"),  " 2023 WORKBOOK[A-Za-z \\.]*"))
names(myfiles23) <- sapply(myfiles23, fixfilename2023)
options(repr.matrix.max.cols=100)

In [8]:
makecountyDF <- function(x) {
  tempDF <- cbind(names(myfiles23)[x], fread(myfiles23[x]))
  names(tempDF)[1] <- "County"
  return(tempDF)
}

In [9]:
myresults23 <- lapply(1:length(myfiles23), makecountyDF)
names(myresults23) <- names(myfiles23)
sapply(myresults23, dim)
head(myresults23[[33]])

Adams,Allen,Bartholomew,Benton,Blackford,Boone,Brown,Carroll,Cass,Clark,Clay,Clinton,Crawford,Daviess,Dearborn,Decatur,Dekalb,Delaware,Dubois,Elkhart,Fayette,Floyd,Fountain,Franklin,Fulton,Gibson,Grant,Greene,Hamilton,Hancock,Harrison,Hendricks,Henry,Howard,Huntington,Jackson,Jasper,Jay,Jefferson,Jennings,Johnson,Knox,Kosciusko,Lagrange,Lake,Laporte,Lawrence,Madison,Marion,Marshall,Martin,Miami,Monroe,Montgomery,Morgan,Newton,Noble,Ohio,Orange,Owen,Parke,Perry,Pike,Porter,Posey,Pulaski,Putnam,Randolph,Ripley,Rush,Scott,Shelby,Spencer,St. Joseph,Starke,Steuben,Sullivan,Switzerland,Tippecanoe,Tipton,Union,Vanderburgh,Vermillion,Vigo,Wabash,Warren,Warrick,Washington,Wayne,Wells,White
19545,164251,38360,7841,8797,36512,15748,21331,25174,61403,23713,23176,11109,22138,32192,17757,31329,59274,34953,87527,14086,40024,14425,17531,17190,21909,36802,26552,142647,41699,27239,76978,28139,43604,22737,26422,21379,15657,20177,20498,67576,31046,52208,27536,241920,64617,31431,83736,344019,30365,10280,22804,55234,23349,41965,11327,26747,3512,21635,17563,20045,16413,20175,82973,19651,17850,25882,18365,18395,11958,15154,29893,22289,118696,19792,38986,1048575,8311,68389,12286,6404,79365,12608,59798,19352,10346,43022,19419,35067,16259,21598
20,20,20,20,20,20,17,20,20,17,18,20,21,20,20,20,20,20,21,20,20,20,17,17,20,22,20,22,20,20,17,20,14,20,17,20,17,20,20,20,20,22,20,20,15,20,20,20,17,20,20,20,20,20,20,20,20,20,20,22,16,22,20,20,22,20,20,20,20,17,20,20,21,20,20,20,20,20,20,20,20,20,20,18,20,20,22,20,20,20,20


County,State Parcel Number,State District ID,Township,Neighborhood Code,Property Class,State Township ID,Land AV,Impr AV,Total AV,WIP Land AV,WIP Impr AV,WIP Total AV,Review Group
<chr>,<chr>,<int>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Henry,33-01-25-000-101.000-005,5,FALL CREEK TOWNSHIP,0003C000-005,100,3,"$26,800",$0,"$26,800","$33,900",$0,"$33,900",2023
Henry,33-01-25-000-101.001-005,5,FALL CREEK TOWNSHIP,0003C000-005,100,3,"$26,500",$0,"$26,500","$33,600",$0,"$33,600",2023
Henry,33-01-25-000-101.002-005,5,FALL CREEK TOWNSHIP,0003C000-005,100,3,"$25,300",$0,"$25,300","$32,100",$0,"$32,100",2023
Henry,33-01-25-000-101.003-005,5,FALL CREEK TOWNSHIP,0003C000-005,100,3,"$28,300",$0,"$28,300","$35,900",$0,"$35,900",2023
Henry,33-01-25-000-101.004-005,5,FALL CREEK TOWNSHIP,0003C000-005,100,3,"$27,200",$0,"$27,200","$34,500",$0,"$34,500",2023
Henry,33-01-25-000-102.000-005,5,FALL CREEK TOWNSHIP,0003C000-005,101,3,"$28,400","$155,600","$184,000","$41,600","$190,900","$232,500",2023


In [10]:
# Creating 2023 Statewide Workbook
# get parcel numbers
myparcelnumbers <- lapply(1:91, function(x) myresults23[[x]]$"Parcel Number")
which(sapply(myparcelnumbers, is.null))
myparcelnumbers[[33]] <- myresults23[[33]]$"State Parcel Number"
table(sapply(myparcelnumbers, class))
# get prior year property class
myprioryearpropertyclass <- lapply(1:91, function(x) myresults23[[x]]$"Prior Year Property Class")
which(sapply(myprioryearpropertyclass, is.null))
myprioryearpropertyclass[[7]] <- myresults23[[7]]$"Prior  Year Property Class"
myprioryearpropertyclass[[10]] <- myresults23[[10]]$"Prior  Year Property Class"
myprioryearpropertyclass[[33]] <- myresults23[[33]]$"Property Class"
table(sapply(myprioryearpropertyclass, class))
# get current year property class
mycurrentyearpropertyclass <- lapply(1:91, function(x) myresults23[[x]]$"Current Year Property Class")
which(sapply(mycurrentyearpropertyclass, is.null))
mycurrentyearpropertyclass[[33]] <- myresults23[[33]]$"Property Class"
table(sapply(mycurrentyearpropertyclass, class))
myprioryeartotalAV <- lapply(1:91, function(x) as.numeric(gsub(",", "", myresults23[[x]]$"Prior Year Total AV")))
which(sapply(mycurrentyearpropertyclass, is.null))
which(is.na(myprioryeartotalAV))
myprioryeartotalAV[[33]] <- as.numeric(gsub("\\$", "", gsub(",", "", myresults23[[33]]$"Total AV")))
table(sapply(myprioryeartotalAV, class))
# get current year total AV
mycurrentyeartotalAV <- lapply(1:91, function(x) as.numeric(gsub(",", "", myresults23[[x]]$"Current Year Total AV")))
which(sapply(mycurrentyeartotalAV, is.null))
which(is.na(myprioryeartotalAV))
mycurrentyeartotalAV[[33]] <- as.numeric(gsub("\\$", "", gsub(",", "", myresults23[[33]]$"WIP Total AV")))
table(sapply(mycurrentyeartotalAV, class))

[1] 33


character 
       91 

[1]  7 10 33


integer 
     91 

[1] 33

...

integer(0)


numeric 
     91 

integer(0)

integer(0)


numeric 
     91 

In [11]:
# 2023 Statewide Workbook
combinedwkbk23 = data.frame(
  County = character(),
  ParcelNumber = character(),
  PriorPropertyClass = integer(),
  CurrentPropertyClass = integer(), 
  PriorAV = numeric(),
  CurrentAV = numeric()
)
for (i in 1:91) {
    newdata = data.frame(
      County = myresults23[[i]]$'County',
      ParcelNumber = myparcelnumbers[[i]],
      PriorPropertyClass = myprioryearpropertyclass[[i]],
      CurrentPropertyClass = mycurrentyearpropertyclass[[i]], 
      PriorAV = myprioryeartotalAV[[i]],
      CurrentAV = mycurrentyeartotalAV[[i]]
    )
    combinedwkbk23 = rbind(combinedwkbk23,newdata)
}
head(combinedwkbk23)

,County,ParcelNumber,PriorPropertyClass,CurrentPropertyClass,PriorAV,CurrentAV
,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>
1,Adams,01-01-01-100-001.000-012,101,101,216600,231500
2,Adams,01-01-01-100-002.000-012,100,100,55100,69700
3,Adams,01-01-01-100-002.500-012,100,100,72600,92000
4,Adams,01-01-01-100-004.000-012,685,685,232000,244100
5,Adams,01-01-01-100-005.000-012,610,610,0,0
6,Adams,01-01-01-100-006.000-012,610,610,0,0


In [58]:
combinedexc23 = data.frame(
  County = character(),
  ParcelNumber = character(),
  PropertyClass = integer(),
  PostedAV = numeric(),
  CurrentAV = numeric(),
  SalePrice = numeric(),
  PostedSaleRatio = numeric(),
  SaleRatio = numeric(),
  PostedExplanation = character(),
  Explanation = character(),
  Reclassified = logical()
)
for (county in names(myfiles23)) {
    ctywkbk = combinedwkbk23[combinedwkbk23$County == county,]
    if (gsub(" ", "", county) %in% c('Warrick','Lagrange')) {
        next
        ##Warrick 2023 Sales Reconciliation is incomplete - No AV included
        ##LaGrange 2023 Sales Reconciliation is incomplete - No Explanations included
    }
    if (gsub(" ", "", county) == 'Dekalb') {
        excludedbook = read_excel(paste0('tippecanoe/data/2023 Ratio Study - Sales Reconciliation Submissions/DeKalb County Sales Reconciliation.xlsx'))  
    } else if (gsub(" ", "", county) == 'Lagrange') {
        excludedbook = read_excel(paste0('tippecanoe/data/2023 Ratio Study - Sales Reconciliation Submissions/LaGrange County Sales Reconciliation.xlsx'))  
    } else if (gsub(" ", "", county) == 'Laporte') {
        excludedbook = read_excel(paste0('tippecanoe/data/2023 Ratio Study - Sales Reconciliation Submissions/LaPorte County Sales Reconciliation.xlsx'))  
    } else if (gsub(" ", "", county) %in% c('St.Joseph')) {
        excludedbook = read_excel(paste0('tippecanoe/data/2023 Ratio Study - Sales Reconciliation Submissions/St. Joseph County Sales Reconciliation.xlsx'))  
    } else if (gsub(" ", "", county) == 'Pike') {
        excludedbook = read_excel(paste0('tippecanoe/data/2023 Ratio Study - Sales Reconciliation Submissions/Pike County Reconciliation.xlsx'))  
    } else if (gsub(" ", "", county) == 'Pulaski') {
        excludedbook = read_excel(paste0('tippecanoe/data/2023 Ratio Study - Sales Reconciliation Submissions/Pulaski County Sales Reconcilation.xlsx'))  
    } else if (gsub(" ", "", county) == 'Miami') {
        excludedbook = read_excel('tippecanoe/data/2023 Ratio Study - Sales Reconciliation Submissions/Miami County Sales Reconciliation.xlsx',sheet='Invalids with explanation')  
    } else if (gsub(" ", "", county) == 'Kosciusko') {
        excludedbook = read_excel('tippecanoe/data/2023 Ratio Study - Sales Reconciliation Submissions/Kosciusko County Sales Reconciliation.xlsx',sheet='W Explanations')
        excludedbook = excludedbook[!is.na(excludedbook$Explanation),]
    } else if (gsub(" ", "", county) == 'Fountain') {
        excludedbook = read_excel('tippecanoe/data/2023 Ratio Study - Sales Reconciliation Submissions/Fountain County Sales Reconciliation.xlsx')
        colnames(excludedbook)[colnames(excludedbook) == '...12'] <- "Explanation"
    } else if (gsub(" ", "", county) == 'Clinton') {
        excludedbook = read_excel('tippecanoe/data/2023 Ratio Study - Sales Reconciliation Submissions/Clinton County Sales Reconciliation.xlsx',sheet='invalids with explanations')
        excludedbook$Explanation[is.na(excludedbook$Explanation)] = excludedbook$'Sale Ratio'[is.na(excludedbook$Explanation)]
    } else {
        excludedbook = read_excel(paste0('tippecanoe/data/2023 Ratio Study - Sales Reconciliation Submissions/',gsub(" ", "", county),' County Sales Reconciliation.xlsx'))
    }
    if (gsub(" ", "", county) %in% c('Jennings','Johnson','Scott')) {
        colnames(excludedbook)[colnames(excludedbook) == "FormattedParcelNumber"] <- "Formatted Parcel Number"
        colnames(excludedbook)[colnames(excludedbook) == "PropertyClass"] <- "Property Class"
        colnames(excludedbook)[colnames(excludedbook) == "TotalAV"] <- "Total AV"
        colnames(excludedbook)[colnames(excludedbook) == "SalePrice"] <- "Sale Price"
        colnames(excludedbook)[colnames(excludedbook) == "SaleRatio"] <- "Sale Ratio"
    }
    if (gsub(" ", "", county) %in% c('Bartholomew','Blackford','Carroll','Dearborn','Delaware','Fayette'
                                     ,'Fulton','Gibson','Henry','Jasper','Jay','Kosciusko','Madison'
                                    ,'Miami','Ohio','Perry','Posey','Randolph','Rush','Scott','Sullivan'
                                    ,'Switzerland','Vermillion')) {
        excludedbook$'Sale Ratio' = as.numeric(excludedbook$'Total AV') / as.numeric(excludedbook$'Sale Price')
    }
    if (gsub(" ", "", county) %in% c('Laporte')) {
        colnames(excludedbook)[colnames(excludedbook) == "Sales Price"] <- "Sale Price"
        excludedbook$'Sale Ratio' = as.numeric(excludedbook$'Total AV') / as.numeric(excludedbook$'Sale Price')
    }
    if (gsub(" ", "", county) %in% c('Owen')) {
        colnames(excludedbook)[colnames(excludedbook) == 'Formatted Parcel Number...6'] <- "Formatted Parcel Number"
    }
    if (gsub(" ", "", county) %in% c('Parke')) {
        colnames(excludedbook)[colnames(excludedbook) == 'Formatted Parcel Number...6'] <- "Formatted Parcel Number"
        excludedbook$'Sale Ratio' = as.numeric(excludedbook$'Total AV') / as.numeric(excludedbook$'Sale Price')
    }
    if (gsub(" ", "", county) %in% c('Bartholomew')) {
        excludedbook$Explanation[is.na(excludedbook$Explanation)] = excludedbook$Comment[is.na(excludedbook$Explanation)]
    }
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("included", Explanation, ignore.case = TRUE) ~ "ON RATIO STUDY",
        grepl("ratio study", Explanation, ignore.case = TRUE) ~ "ON RATIO STUDY",
        grepl("used", Explanation, ignore.case = TRUE) ~ "ON RATIO STUDY",
        grepl("in study", Explanation, ignore.case = TRUE) ~ "ON RATIO STUDY",
        grepl("validsale", Explanation, ignore.case = TRUE) ~ "ON RATIO STUDY",
        grepl("-valid sale", Explanation, ignore.case = TRUE) ~ "ON RATIO STUDY",
        grepl("formatted tab", Explanation, ignore.case = TRUE) ~ "ON RATIO STUDY",
        Explanation == "VALID" ~ "ON RATIO STUDY",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook = excludedbook[excludedbook$Explanation != "ON RATIO STUDY", ]
    postedexp <- ifelse(is.na(excludedbook$Explanation), "Blank", excludedbook$Explanation)
    if (gsub(" ", "", county) %in% c('Marion','Morgan')) {
        excludedbook$Explanation[is.na(excludedbook$Explanation)] = 'NO REASON PROVIDED'
    }
    if (gsub(" ", "", county) %in% c('Hancock')) {
        excludedbook$Explanation[is.na(excludedbook$Explanation)] = 'NO REASON PROVIDED'
    }
    if (!("Explanation" %in% colnames(excludedbook))) {
        message("Column 'Explanation' not found in ", county)
    }
    if (!("Formatted Parcel Number" %in% colnames(excludedbook))) {
        message("Column 'Formatted Parcel Number' not found in ", county)
    }
    if (!("Total AV" %in% colnames(excludedbook))) {
        message("Column 'Total AV' not found in ", county)
    }
    if (!("Sale Price" %in% colnames(excludedbook))) {
        message("Column 'Sale Price' not found in ", county)
    }
    if (!("Sale Ratio" %in% colnames(excludedbook))) {
        message("Column 'Sale Ratio' not found in ", county)
    }
    if (!("Property Class" %in% colnames(excludedbook))) {
        message("Column 'Property Class' not found in ", county)
    }
    
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("golf", Explanation, ignore.case = TRUE) ~ "GOLF COURSE",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("LEASE", Explanation, ignore.case = TRUE) ~ "LEASE",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("campground", Explanation, ignore.case = TRUE) ~ "CAMPGROUND",
        grepl("JELLYSTONE", Explanation, ignore.case = TRUE) ~ "CAMPGROUND",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("common", Explanation, ignore.case = TRUE) ~ "COMMON GROUND",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("permit", Explanation, ignore.case = TRUE) ~ "BUILDING PERMIT",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("MOBILE", Explanation, ignore.case = TRUE) ~ "MOBILE HOME",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("special use", Explanation, ignore.case = TRUE) ~ "SPECIAL USE",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("Adja", Explanation, ignore.case = TRUE) ~ "ADJACENT OWNER",
        grepl("Adjo", Explanation, ignore.case = TRUE) ~ "ADJACENT OWNER",
        grepl("next door", Explanation, ignore.case = TRUE) ~ "ADJACENT OWNER",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("duplicate", Explanation, ignore.case = TRUE) ~ "DUPLICATE",
        grepl("dup sumb", Explanation, ignore.case = TRUE) ~ "DUPLICATE",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("PARTIAL", Explanation, ignore.case = TRUE) ~ "PARTIAL INTEREST",
        grepl("Percent int", Explanation, ignore.case = TRUE) ~ "PARTIAL INTEREST",
        grepl("half int", Explanation, ignore.case = TRUE) ~ "PARTIAL INTEREST",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("INVEST", Explanation, ignore.case = TRUE) ~ "INVESTMENT/RENTAL",
        grepl("RENT", Explanation, ignore.case = TRUE) ~ "INVESTMENT/RENTAL",
        grepl("INCOME", Explanation, ignore.case = TRUE) ~ "INVESTMENT/RENTAL",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("sold as is", Explanation, ignore.case = TRUE) ~ "SOLD AS IS",
        grepl("as-is", Explanation, ignore.case = TRUE) ~ "SOLD AS IS",
        grepl("as is", Explanation, ignore.case = TRUE) ~ "SOLD AS IS",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("MIXED", Explanation, ignore.case = TRUE) ~ "RELATED ASSETS",
        grepl("PORTFOLIO", Explanation, ignore.case = TRUE) ~ "RELATED ASSETS",
        grepl("mix use", Explanation, ignore.case = TRUE) ~ "RELATED ASSETS",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("apt", Explanation, ignore.case = TRUE) ~ "APARTMENT",
        grepl("apartment", Explanation, ignore.case = TRUE) ~ "APARTMENT",
        grepl("condo", Explanation, ignore.case = TRUE) ~ "APARTMENT",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("OUTSIDE", Explanation, ignore.case = TRUE) ~ "OUTSIDE TIMEFRAME",
        grepl("time frame", Explanation, ignore.case = TRUE) ~ "OUTSIDE TIMEFRAME",
        grepl("future sale", Explanation, ignore.case = TRUE) ~ "OUTSIDE TIMEFRAME",
        grepl("conveyance date", Explanation, ignore.case = TRUE) ~ "OUTSIDE TIMEFRAME",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("CONSTRUCTION", Explanation, ignore.case = TRUE) ~ "NEW CONSTRUCTION",
        grepl("new plat", Explanation, ignore.case = TRUE) ~ "NEW CONSTRUCTION",
        grepl("new home", Explanation, ignore.case = TRUE) ~ "NEW CONSTRUCTION",
        grepl("new house", Explanation, ignore.case = TRUE) ~ "NEW CONSTRUCTION",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("prior", Explanation, ignore.case = TRUE) ~ "CHANGES PRIOR",
        grepl("before sale", Explanation, ignore.case = TRUE) ~ "CHANGES PRIOR",
        grepl("changes before", Explanation, ignore.case = TRUE) ~ "CHANGES PRIOR",
        grepl("changebefore", Explanation, ignore.case = TRUE) ~ "CHANGES PRIOR",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("CLASS", Explanation, ignore.case = TRUE) ~ "INVALID PROPERTY CLASS",
        grepl("commercial", Explanation, ignore.case = TRUE) ~ "INVALID PROPERTY CLASS",
        grepl("not com", Explanation, ignore.case = TRUE) ~ "INVALID PROPERTY CLASS",
        grepl("notcom", Explanation, ignore.case = TRUE) ~ "INVALID PROPERTY CLASS",
        grepl("type changed", Explanation, ignore.case = TRUE) ~ "INVALID PROPERTY CLASS",
        grepl("industrial", Explanation, ignore.case = TRUE) ~ "INVALID PROPERTY CLASS",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("LIMITED", Explanation, ignore.case = TRUE) ~ "LIMITED INFO",
        grepl("INFO", Explanation, ignore.case = TRUE) ~ "LIMITED INFO",
        grepl("NAME CHANGE", Explanation, ignore.case = TRUE) ~ "LIMITED INFO",
        grepl("No response", Explanation, ignore.case = TRUE) ~ "LIMITED INFO",
        grepl("unable to verify", Explanation, ignore.case = TRUE) ~ "LIMITED INFO",
        grepl("no sale price", Explanation, ignore.case = TRUE) ~ "LIMITED INFO",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("sufficient", Explanation, ignore.case = TRUE) ~ "INSUFFICIENT TIME ON MARKET",
        grepl("TIME ON MARKET", Explanation, ignore.case = TRUE) ~ "INSUFFICIENT TIME ON MARKET",
        grepl("TIME ON THE MARKET", Explanation, ignore.case = TRUE) ~ "INSUFFICIENT TIME ON MARKET",
        grepl("same day", Explanation, ignore.case = TRUE) ~ "INSUFFICIENT TIME ON MARKET",
        grepl("1 day", Explanation, ignore.case = TRUE) ~ "INSUFFICIENT TIME ON MARKET",
        grepl("one day", Explanation, ignore.case = TRUE) ~ "INSUFFICIENT TIME ON MARKET",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("trend", Explanation, ignore.case = TRUE) ~ "NOT ENOUGH SALES FOR TRENDING",
        grepl("not enough", Explanation, ignore.case = TRUE) ~ "NOT ENOUGH SALES FOR TRENDING",
        grepl("to study", Explanation, ignore.case = TRUE) ~ "NOT ENOUGH SALES FOR TRENDING",
        grepl("to do study", Explanation, ignore.case = TRUE) ~ "NOT ENOUGH SALES FOR TRENDING",
        grepl("lack of sales", Explanation, ignore.case = TRUE) ~ "NOT ENOUGH SALES FOR TRENDING",
        grepl("not using", Explanation, ignore.case = TRUE) ~ "NOT ENOUGH SALES FOR TRENDING",
        grepl("sales for study", Explanation, ignore.case = TRUE) ~ "NOT ENOUGH SALES FOR TRENDING",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("RELIGIO", Explanation, ignore.case = TRUE) ~ "SOLD TO EXEMPT ORGANIZATION",
        grepl("PROFIT", Explanation, ignore.case = TRUE) ~ "SOLD TO EXEMPT ORGANIZATION",
        grepl("GOV", Explanation, ignore.case = TRUE) ~ "SOLD TO EXEMPT ORGANIZATION",
        grepl("CHARIT", Explanation, ignore.case = TRUE) ~ "SOLD TO EXEMPT ORGANIZATION",
        grepl("EDUCATI", Explanation, ignore.case = TRUE) ~ "SOLD TO EXEMPT ORGANIZATION",
        grepl("school", Explanation, ignore.case = TRUE) ~ "SOLD TO EXEMPT ORGANIZATION",
        grepl("exempt", Explanation, ignore.case = TRUE) ~ "SOLD TO EXEMPT ORGANIZATION",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("sold for less than $", Explanation, ignore.case = TRUE) ~ "LOW SALE PRICE",
        grepl("\\$\\w+,*\\w+\\s*SALE", Explanation, ignore.case = TRUE) ~ "LOW SALE PRICE",
        grepl("low sale price", Explanation, ignore.case = TRUE) ~ "LOW SALE PRICE",
        grepl("ZERO sales price", Explanation, ignore.case = TRUE) ~ "LOW SALE PRICE",
        grepl("sale amount less", Explanation, ignore.case = TRUE) ~ "LOW SALE PRICE",
        grepl("sale less th", Explanation, ignore.case = TRUE) ~ "LOW SALE PRICE",
        grepl("sold for under", Explanation, ignore.case = TRUE) ~ "LOW SALE PRICE",
        grepl("SOLD under", Explanation, ignore.case = TRUE) ~ "LOW SALE PRICE",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("in use", Explanation, ignore.case = TRUE) ~ "CHANGE IN USE",
        grepl("PRIMARY USE", Explanation, ignore.case = TRUE) ~ "CHANGE IN USE",
        grepl("USE CHANGE", Explanation, ignore.case = TRUE) ~ "CHANGE IN USE",
        grepl("CHANGE IN", Explanation, ignore.case = TRUE) ~ "CHANGE IN USE",
        grepl("CHANGE UN", Explanation, ignore.case = TRUE) ~ "CHANGE IN USE",
        grepl("changed use", Explanation, ignore.case = TRUE) ~ "CHANGE IN USE",
        grepl("change of use", Explanation, ignore.case = TRUE) ~ "CHANGE IN USE",
        grepl("changeinuse", Explanation, ignore.case = TRUE) ~ "CHANGE IN USE",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("vac", Explanation, ignore.case = TRUE) ~ "VACANT LAND",
        grepl("v/L", Explanation, ignore.case = TRUE) ~ "VACANT LAND",
        grepl("INACTIVE", Explanation, ignore.case = TRUE) ~ "VACANT LAND",
        grepl("abandoned", Explanation, ignore.case = TRUE) ~ "VACANT LAND",
        grepl("bare", Explanation, ignore.case = TRUE) ~ "VACANT LAND",
        Explanation == "VA" ~ "VACANT LAND",
        grepl("HOMESITE", Explanation, ignore.case = TRUE) ~ "VACANT LAND",
        grepl("dwelling", Explanation, ignore.case = TRUE) ~ "VACANT LAND",
        grepl("residential structure", Explanation, ignore.case = TRUE) ~ "VACANT LAND",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("dev", Explanation, ignore.case = TRUE) ~ "BUSINESS TRANSACTION",
        grepl("BUILDER", Explanation, ignore.case = TRUE) ~ "BUSINESS TRANSACTION",
        grepl("BUSINESS", Explanation, ignore.case = TRUE) ~ "BUSINESS TRANSACTION",
        grepl("CONSOLID", Explanation, ignore.case = TRUE) ~ "BUSINESS TRANSACTION",
        grepl("REBUILD", Explanation, ignore.case = TRUE) ~ "BUSINESS TRANSACTION",
        grepl("llc", Explanation, ignore.case = TRUE) ~ "BUSINESS TRANSACTION",
        grepl("holdings company", Explanation, ignore.case = TRUE) ~ "BUSINESS TRANSACTION",
        grepl("lending inst", Explanation, ignore.case = TRUE) ~ "BUSINESS TRANSACTION",
        grepl("reo", Explanation, ignore.case = TRUE) ~ "BUSINESS TRANSACTION",
        grepl("corporat", Explanation, ignore.case = TRUE) ~ "BUSINESS TRANSACTION",
        grepl("MERGER", Explanation, ignore.case = TRUE) ~ "BUSINESS TRANSACTION",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("not exist", Explanation, ignore.case = TRUE) ~ "INVALID PARCEL NUMBER",
        grepl("ACTIVE PARCEL", Explanation, ignore.case = TRUE) ~ "INVALID PARCEL NUMBER",
        grepl("wrong parcel", Explanation, ignore.case = TRUE) ~ "INVALID PARCEL NUMBER",
        grepl("no longer a parcel", Explanation, ignore.case = TRUE) ~ "INVALID PARCEL NUMBER",
        grepl("no longer exists", Explanation, ignore.case = TRUE) ~ "INVALID PARCEL NUMBER",
        grepl("retired", Explanation, ignore.case = TRUE) ~ "INVALID PARCEL NUMBER",
        grepl("is not active", Explanation, ignore.case = TRUE) ~ "INVALID PARCEL NUMBER",
        grepl("not an active", Explanation, ignore.case = TRUE) ~ "INVALID PARCEL NUMBER",
        grepl("deleted parcel", Explanation, ignore.case = TRUE) ~ "INVALID PARCEL NUMBER",
        grepl("parcelisnolonger", Explanation, ignore.case = TRUE) ~ "INVALID PARCEL NUMBER",
        grepl("no longer active", Explanation, ignore.case = TRUE) ~ "INVALID PARCEL NUMBER",
        grepl("no loner", Explanation, ignore.case = TRUE) ~ "INVALID PARCEL NUMBER",
        TRUE ~ Explanation # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("BANKRUP", Explanation, ignore.case = TRUE) ~ "FORCED SALE",
        grepl("FORECLO", Explanation, ignore.case = TRUE) ~ "FORCED SALE",
        grepl("DIVORCE", Explanation, ignore.case = TRUE) ~ "FORCED SALE",
        grepl("forced", Explanation, ignore.case = TRUE) ~ "FORCED SALE",
        grepl("distressed", Explanation, ignore.case = TRUE) ~ "FORCED SALE",
        grepl("duress", Explanation, ignore.case = TRUE) ~ "FORCED SALE",
        grepl("hardship", Explanation, ignore.case = TRUE) ~ "FORCED SALE",
        grepl("court", Explanation, ignore.case = TRUE) ~ "FORCED SALE",
        grepl("death", Explanation, ignore.case = TRUE) ~ "FORCED SALE",
        grepl("compulsory", Explanation, ignore.case = TRUE) ~ "FORCED SALE",
        grepl("repo", Explanation, ignore.case = TRUE) ~ "FORCED SALE",
        grepl("judicial", Explanation, ignore.case = TRUE) ~ "FORCED SALE",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("AFTER SALE", Explanation, ignore.case = TRUE) ~ "CHANGES AFTER",
        grepl("SIGNIFICANT", Explanation, ignore.case = TRUE) ~ "CHANGES AFTER",
        grepl("PHYSICAL CHANGE", Explanation, ignore.case = TRUE) ~ "CHANGES AFTER",
        grepl("remodel", Explanation, ignore.case = TRUE) ~ "CHANGES AFTER",
        grepl("reno", Explanation, ignore.case = TRUE) ~ "CHANGES AFTER",
        grepl("changes after", Explanation, ignore.case = TRUE) ~ "CHANGES AFTER",
        grepl("improve", Explanation, ignore.case = TRUE) ~ "CHANGES AFTER",
        grepl("add", Explanation, ignore.case = TRUE) ~ "CHANGES AFTER",
        grepl("updat", Explanation, ignore.case = TRUE) ~ "CHANGES AFTER",
        grepl("post sale", Explanation, ignore.case = TRUE) ~ "CHANGES AFTER",
        grepl("since last assessment", Explanation, ignore.case = TRUE) ~ "CHANGES AFTER",
        grepl("after the sale", Explanation, ignore.case = TRUE) ~ "CHANGES AFTER",
        grepl("changeafter", Explanation, ignore.case = TRUE) ~ "CHANGES AFTER",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("twice", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        grepl("2X", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        grepl("more than", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        grepl("more then", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        grepl("three times", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        grepl("resold", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        grepl("re-sold", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        grepl("2 sales", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        grepl("first of two", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        grepl("2 times", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        grepl("4 times", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        grepl("3 times", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        grepl("second sale", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        grepl("also sold", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        grepl("again", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        grepl("later date", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        grepl("same parcel sold", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        grepl("months later", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("family", Explanation, ignore.case = TRUE) ~ "PRE-EXISTING RELATIONSHIP",
        grepl("relat", Explanation, ignore.case = TRUE) ~ "PRE-EXISTING RELATIONSHIP",
        grepl("LATive", Explanation, ignore.case = TRUE) ~ "PRE-EXISTING RELATIONSHIP",
        grepl("AFFILIA", Explanation, ignore.case = TRUE) ~ "PRE-EXISTING RELATIONSHIP",
        grepl("SAME OWNER", Explanation, ignore.case = TRUE) ~ "PRE-EXISTING RELATIONSHIP",
        grepl("S LENGTH", Explanation, ignore.case = TRUE) ~ "PRE-EXISTING RELATIONSHIP",
        grepl("IS TENANT", Explanation, ignore.case = TRUE) ~ "PRE-EXISTING RELATIONSHIP",
        grepl("same company", Explanation, ignore.case = TRUE) ~ "PRE-EXISTING RELATIONSHIP",
        grepl("inherit", Explanation, ignore.case = TRUE) ~ "PRE-EXISTING RELATIONSHIP",
        grepl("same person", Explanation, ignore.case = TRUE) ~ "PRE-EXISTING RELATIONSHIP",
        grepl("same entity", Explanation, ignore.case = TRUE) ~ "PRE-EXISTING RELATIONSHIP",
        grepl("friend", Explanation, ignore.case = TRUE) ~ "PRE-EXISTING RELATIONSHIP",
        grepl("arms len", Explanation, ignore.case = TRUE) ~ "PRE-EXISTING RELATIONSHIP",
        grepl("arms-length", Explanation, ignore.case = TRUE) ~ "PRE-EXISTING RELATIONSHIP",
        grepl("ated org", Explanation, ignore.case = TRUE) ~ "PRE-EXISTING RELATIONSHIP",
        grepl("survivorship", Explanation, ignore.case = TRUE) ~ "PRE-EXISTING RELATIONSHIP",
        grepl("probate", Explanation, ignore.case = TRUE) ~ "PRE-EXISTING RELATIONSHIP",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("AUCTION", Explanation, ignore.case = TRUE) ~ "UNIQUE SALE",
        grepl("ESTATE", Explanation, ignore.case = TRUE) ~ "UNIQUE SALE",
        grepl("TRUST", Explanation, ignore.case = TRUE) ~ "UNIQUE SALE",
        grepl("BANK", Explanation, ignore.case = TRUE) ~ "UNIQUE SALE",
        grepl("CASH", Explanation, ignore.case = TRUE) ~ "UNIQUE SALE",
        grepl("QUIT", Explanation, ignore.case = TRUE) ~ "UNIQUE SALE",
        grepl("QC", Explanation, ignore.case = TRUE) ~ "UNIQUE SALE",
        grepl("TAX", Explanation, ignore.case = TRUE) ~ "UNIQUE SALE",
        grepl("SHORT SALE", Explanation, ignore.case = TRUE) ~ "UNIQUE SALE",
        grepl("CONTRACT", Explanation, ignore.case = TRUE) ~ "UNIQUE SALE",
        grepl("HUD", Explanation, ignore.case = TRUE) ~ "UNIQUE SALE",
        grepl("GUARDIANSHIP", Explanation, ignore.case = TRUE) ~ "UNIQUE SALE",
        grepl("sheriff", Explanation, ignore.case = TRUE) ~ "UNIQUE SALE",
        grepl("lc payoff", Explanation, ignore.case = TRUE) ~ "UNIQUE SALE",
        grepl("financ", Explanation, ignore.case = TRUE) ~ "UNIQUE SALE",
        grepl("trade", Explanation, ignore.case = TRUE) ~ "UNIQUE SALE",
        grepl("arbitration", Explanation, ignore.case = TRUE) ~ "UNIQUE SALE",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("FIXER", Explanation, ignore.case = TRUE) ~ "POOR CONDITION",
        grepl("flip", Explanation, ignore.case = TRUE) ~ "POOR CONDITION",
        grepl("CONDITION", Explanation, ignore.case = TRUE) ~ "POOR CONDITION",
        grepl("liveable", Explanation, ignore.case = TRUE) ~ "POOR CONDITION",
        grepl("uninhabit", Explanation, ignore.case = TRUE) ~ "POOR CONDITION",
        grepl("rehab", Explanation, ignore.case = TRUE) ~ "POOR CONDITION",
        grepl("fire", Explanation, ignore.case = TRUE) ~ "POOR CONDITION",
        grepl("remov", Explanation, ignore.case = TRUE) ~ "POOR CONDITION",
        grepl("bad shape", Explanation, ignore.case = TRUE) ~ "POOR CONDITION",
        grepl("demo", Explanation, ignore.case = TRUE) ~ "POOR CONDITION",
        grepl("needs rep", Explanation, ignore.case = TRUE) ~ "POOR CONDITION",
        grepl("disrepair", Explanation, ignore.case = TRUE) ~ "POOR CONDITION",
        grepl("contamination", Explanation, ignore.case = TRUE) ~ "POOR CONDITION",
        grepl("condemn", Explanation, ignore.case = TRUE) ~ "POOR CONDITION",
        grepl("rough shape", Explanation, ignore.case = TRUE) ~ "POOR CONDITION",
        grepl("burnt", Explanation, ignore.case = TRUE) ~ "POOR CONDITION",
        grepl("need of repair", Explanation, ignore.case = TRUE) ~ "POOR CONDITION",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("mul", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("MPS", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("parcel sale", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("package sale", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("combin", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("with another", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("split", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("2 properties", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("partition", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("2 parcel", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("contiguous", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("two parcel", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("2 part", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("two separate", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("group of", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("prices were arbit", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("4 parcels", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("bulk property", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("shares sale price", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("sold with", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("several", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        TRUE ~ Explanation
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("open market", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("closed sale", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("hit the market", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("never hit", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("never listed", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("NOT listed", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("NOT on", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("fsbo", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("buyer approached", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("private", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("never on market", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("for sale by owner", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("personal rep", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("word of mouth", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("sold by owner", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("market time", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("expos", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("advertise", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("off market", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("not marketed", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("out of market", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("no listing", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("non marketed", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("never on the market", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
     excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("assessed", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("less than", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("discount", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("motivated", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("priced to sell", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("concessions", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("price drop", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("typical", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("below market", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("indicative", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("considerably", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("outlier", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("market variab", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("reflect", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("reduced", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("under market", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("neighborhood", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("represent", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("overpaid", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("intangible", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("per. prop", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("nontangible", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("sell off", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("goodwill", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("personal prop", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("person. prop", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("seller paid", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("appraisal completed", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("value set on appeal", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("asking price", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("recent sale", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("INCORRECT SALE PRICE", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("over av", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("errors in assessment", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("statistical range", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("unusual for market", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("yard items", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("knowledge of sale", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("belowmarket", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("undermarket", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("per.prop", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("furnish", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("bid", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("excessive price", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("PP ", Explanation, ignore.case = FALSE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("includes pp", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("% AV", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("% of AV", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("max depreciation", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("out building", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("outbuilding", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("UNKNOWLEDGEABLE", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("AG", Explanation, ignore.case = TRUE) ~ "AG LAND",
        grepl("farm", Explanation, ignore.case = TRUE) ~ "AG LAND",
        grepl("barn", Explanation, ignore.case = TRUE) ~ "AG LAND",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        Explanation == "INVALID" ~ "NO REASON PROVIDED",
        Explanation == "Invalid" ~ "NO REASON PROVIDED",
        Explanation == "invalid" ~ "NO REASON PROVIDED",
        Explanation == "NOT VALID" ~ "NO REASON PROVIDED",
        Explanation == "not valid" ~ "NO REASON PROVIDED",
        Explanation == "INVALID-" ~ "NO REASON PROVIDED",
        Explanation == "invalid-" ~ "NO REASON PROVIDED",
        Explanation == "N/A" ~ "NO REASON PROVIDED",
        Explanation == " " ~ "NO REASON PROVIDED",
        grepl("other", Explanation, ignore.case = TRUE) ~ "NO REASON PROVIDED",
        grepl("do not use", Explanation, ignore.case = TRUE) ~ "NO REASON PROVIDED",
        TRUE ~ Explanation
      ))
        excluded_indices <- match(excludedbook$'Formatted Parcel Number',ctywkbk$ParcelNumber)
        postedAV = as.numeric(excludedbook$'Total AV')
        excludedbook$'Total AV'[!is.na(excluded_indices)] <- ctywkbk$CurrentAV[excluded_indices[!is.na(excluded_indices)]]
        newdata = data.frame(
          County = rep(county,nrow(excludedbook)),
          ParcelNumber = excludedbook$'Formatted Parcel Number',
          PropertyClass = excludedbook$'Property Class',
          PostedAV = postedAV,
          CurrentAV = as.numeric(excludedbook$'Total AV'),
          SalePrice = as.numeric(excludedbook$'Sale Price'),
          PostedSaleRatio = postedAV / as.numeric(excludedbook$'Sale Price'),
          SaleRatio = as.numeric(excludedbook$'Total AV') / as.numeric(excludedbook$'Sale Price'),
          PostedExplanation = postedexp,
          Explanation = excludedbook$Explanation,
          Reclassified = !(postedexp == excludedbook$Explanation)
        )
        combinedexc23 = rbind(combinedexc23,newdata)
}
head(combinedexc23)

New names:
* `` -> `...12`
New names:
* `` -> `...12`
Warning message:
"Expecting numeric in I2 / R2C9: got a date"
Warning message:
"Expecting numeric in I3 / R3C9: got a date"
Warning message:
"Expecting numeric in I4 / R4C9: got a date"
Warning message:
"Expecting numeric in I5 / R5C9: got a date"
Warning message:
"Expecting numeric in I6 / R6C9: got a date"
Warning message:
"Expecting numeric in I7 / R7C9: got a date"
Warning message:
"Expecting numeric in I8 / R8C9: got a date"
Warning message:
"Expecting numeric in I9 / R9C9: got a date"
Warning message:
"Expecting numeric in I10 / R10C9: got a date"
Warning message:
"Expecting numeric in I11 / R11C9: got a date"
Warning message:
"Expecting numeric in I12 / R12C9: got a date"
Warning message:
"Expecting numeric in I13 / R13C9: got a date"
Warning message:
"Expecting numeric in I14 / R14C9: got a date"
Warning message:
"Expecting numeric in I15 / R15C9: got a date"
Warning message:
"Expecting numeric in I16 / R16C9: got a date"
Wa

,County,ParcelNumber,PropertyClass,PostedAV,CurrentAV,SalePrice,PostedSaleRatio,SaleRatio,PostedExplanation,Explanation,Reclassified
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<lgl>
1,Adams,01-05-03-200-010.000-022,510,77100,95800,134000,0.5753731,0.7149254,Sold Twice,SOLD MORE THAN ONCE IN ONE YEAR,TRUE
2,Adams,01-05-03-113-101.000-022,510,97400,101300,60000,1.6233333,1.6883333,Forced sale other than Foreclosure,FORCED SALE,TRUE
3,Adams,01-02-34-306-004.000-014,510,46200,52300,17000,2.7176471,3.0764706,Significant Physical Change,CHANGES AFTER,TRUE
4,Adams,01-01-13-200-002.000-012,511,314900,367200,554750,0.5676431,0.6619198,Not Typical of the NBHD,DOES NOT REFLECT MARKET VALUE,TRUE
5,Adams,01-05-05-201-001.001-022,500,24100,231400,26000,0.9269231,8.9000000,Significant Physical Change,CHANGES AFTER,TRUE
6,Adams,01-05-03-306-074.000-022,510,60600,68900,20000,3.0300000,3.4450000,Not Listed on Market,NOT LISTED ON OPEN MARKET,TRUE


In [59]:
nrow(combinedexc23[!combinedexc23$Reclassified,])
nrow(combinedexc23)

[1] 1687

[1] 90773

In [14]:
combinedrat23 = data.frame(
  County = character(),
  ParcelNumber = character(),
  PropertyClass = integer(), 
  PostedAV = numeric(),
  CurrentAV = numeric(),
  SalePrice = numeric(),
  PostedSaleRatio = numeric(),
  SaleRatio = numeric()
)
for (county in names(myfiles23)) {
    ctywkbk = combinedwkbk23[combinedwkbk23$County == county,]
    if (gsub(" ", "", county) %in% c('Crawford','Perry','Spencer','Vermillion')) {
        next
        ## Crawford 2023 Approved Ratio Study is .xls file
        ## Perry 2023 Approved Ratio Study is .xls file
        ## Spencer 2023 Approved Ratio Study is .xls file
        ## Vermillion 2023 Approved Ratio Study is .xls file
    }
    
    filepath = paste0('tippecanoe/data/2023 Approved Ratio Studies/',toupper(county),' 2023 RATIO STUDY.xlsx')
    sheets <- excel_sheets(filepath)
    if (!'Formatted' %in% sheets) {
        if (gsub(" ", "", county) %in% c('Fountain')) {
            
        } else {
            stop("Sheet 'Formatted' does not exist in the file: ", county)
        }
    }
    tryCatch({
        if (gsub(" ", "", county) %in% c('Fountain')) {
            ratiobook <- read_excel(filepath, sheet = 'FORMATTED')
        } else {
            ratiobook <- read_excel(filepath, sheet = 'Formatted')
        }
    }, error = function(e) {
        message("Error reading the Excel file: ",county,e$message)
        # Optionally print the structure of the file or a sample
        message("Attempting to read the first few rows.")
        test_read <- read_excel(file_path, sheet = sheets[1])  # Read the first sheet
        print(head(test_read))  # Print the first few rows
    })
    if (gsub(" ", "", county) %in% c('Allen','Clinton','Daviess','Grant','Hendricks','Kosciusko','Laporte'
                                    ,'Lawrence','Marion','Martin','Orange','Parke','Pike','Ripley','Shelby'
                                    ,'Switzerland','Wabash','Wayne','White')) {
        colnames(ratiobook)[colnames(ratiobook) == "ParcelNumber"] <- "Parcel Number"
        colnames(ratiobook)[colnames(ratiobook) == "PropertyClass"] <- "Property Class"
        colnames(ratiobook)[colnames(ratiobook) == "CurrentTotalAV"] <- "Current Total AV"
        colnames(ratiobook)[colnames(ratiobook) == "StudySalePrice"] <- "Study Sale Price"
    }
   if (gsub(" ", "", county) %in% c('Lagrange')) {
        colnames(ratiobook)[colnames(ratiobook) == "Study Sales Price"] <- "Study Sale Price"
    }
    if (gsub(" ", "", county) %in% c('Randolph')) {
        colnames(ratiobook)[colnames(ratiobook) == "ParcelNumber"] <- "Parcel Number"
    }
    if (gsub(" ", "", county) %in% c('Dekalb')) {
        colnames(ratiobook)[colnames(ratiobook) == "Property Class Code"] <- "Property Class"
    }
    if (gsub(" ", "", county) %in% c('Steuben')) {
        colnames(ratiobook)[colnames(ratiobook) == "State Parcel Number"] <- "Parcel Number"
        colnames(ratiobook)[colnames(ratiobook) == "Property Class Code"] <- "Property Class"
    }
    if (!("Parcel Number" %in% colnames(ratiobook))) {
        message("Column 'Parcel Number' not found in ", county)
    }
    if (!("Property Class" %in% colnames(ratiobook))) {
        message("Column 'Property Class' not found in ", county)
    }
    if (!("Current Total AV" %in% colnames(ratiobook))) {
        message("Column 'Current Total AV' not found in ", county)
    }
    if (!("Study Sale Price" %in% colnames(ratiobook))) {
        message("Column 'Study Sale Price' not found in ", county)
    }
    ratio_indices <- match(ratiobook$"Parcel Number",ctywkbk$ParcelNumber)
    postedAV = as.numeric(ratiobook$'Current Total AV')
    ratiobook$'Current Total AV'[!is.na(ratio_indices)] <- ctywkbk$CurrentAV[ratio_indices[!is.na(ratio_indices)]]
    newdata = data.frame(
      County = rep(county,nrow(ratiobook)),
      ParcelNumber = ratiobook$'Parcel Number',
      PropertyClass = ratiobook$'Property Class', 
      PostedAV = postedAV,
      CurrentAV = as.numeric(ratiobook$'Current Total AV'),
      SalePrice = as.numeric(ratiobook$'Study Sale Price'),
      PostedSaleRatio = postedAV / as.numeric(ratiobook$'Study Sale Price'),
      SaleRatio = as.numeric(ratiobook$'Current Total AV') / as.numeric(ratiobook$'Study Sale Price')
    )
    combinedrat23 = rbind(combinedrat23,newdata)
}
head(combinedrat23)

New names:
* `` -> `...16`
New names:
* `` -> `...16`
New names:
* `` -> `...16`
* `` -> `...17`
* `` -> `...18`
* `` -> `...19`
* `` -> `...20`
* `` -> `...21`
* `` -> `...22`
* `` -> `...23`
* `` -> `...24`
* `` -> `...25`
* `` -> `...26`
* `` -> `...27`
* `` -> `...28`
* `` -> `...29`
* `` -> `...30`
* `` -> `...31`
* `` -> `...32`
* `` -> `...33`
* `` -> `...34`
* `` -> `...35`
* `` -> `...36`
* `` -> `...37`
* `` -> `...38`
* `` -> `...39`
* `` -> `...40`
* `` -> `...41`
* `` -> `...42`
* `` -> `...43`
* `` -> `...44`
* `` -> `...45`
* `` -> `...46`
* `` -> `...47`
* `` -> `...48`
* `` -> `...49`
* `` -> `...50`
* `` -> `...51`
* `` -> `...52`
* `` -> `...53`
* `` -> `...54`
* `` -> `...55`
* `` -> `...56`
* `` -> `...57`
* `` -> `...58`
* `` -> `...59`
* `` -> `...60`
* `` -> `...61`
* `` -> `...62`
* `` -> `...63`
* `` -> `...64`
* `` -> `...65`
* `` -> `...66`
* `` -> `...67`
* `` -> `...68`
* `` -> `...69`
* `` -> `...70`
* `` -> `...71`
* `` -> `...72`
* `` -> `...73`
* `` ->

,County,ParcelNumber,PropertyClass,PostedAV,CurrentAV,SalePrice,PostedSaleRatio,SaleRatio
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Adams,01-05-02-301-031.000-022,510,75700,75700,98100,0.7716616,0.7716616
2,Adams,01-05-03-103-060.000-022,510,72700,72700,79800,0.9110276,0.9110276
3,Adams,01-05-02-102-112.000-022,510,131100,131100,158500,0.8271293,0.8271293
4,Adams,01-02-35-100-004.000-013,510,208000,208000,263600,0.7890744,0.7890744
5,Adams,01-11-05-202-044.000-018,510,186600,186600,194200,0.9608651,0.9608651
6,Adams,01-06-08-300-010.000-015,510,173300,173300,213000,0.8136150,0.8136150


In [57]:
combinedexc24 = data.frame(
  County = character(),
  ParcelNumber = character(),
  PropertyClass = integer(), 
  PostedAV = numeric(),
  CurrentAV = numeric(),
  SalePrice = numeric(),
  PostedSaleRatio = numeric(),
  SaleRatio = numeric(),
  PostedExplanation = character(),
  Explanation = character(),
  Reclassified = logical()
)
for (county in names(myfiles24)) {
    ctywkbk = combinedwkbk24[combinedwkbk24$County == county,]
    if (gsub(" ", "", county) %in% c('Kosciusko','Shelby')) {
        next  
        ## Kosciusko 2024 Sales Reconciliation Not Available
        ## Shelby 2024 Sales Reconciliation has no explanations
    }
    if (gsub(" ", "", county) == 'Dekalb') {
        excludedbook = read_excel(paste0('tippecanoe/data/2024 Ratio Study - Sales Reconciliation Submissions/DeKalb County Sales Reconciliation.xlsx'))  
    } else if (gsub(" ", "", county) == 'Lagrange') {
        excludedbook = read_excel(paste0('tippecanoe/data/2024 Ratio Study - Sales Reconciliation Submissions/LaGrange County Sales Reconciliation.xlsx'))
    } else if (gsub(" ", "", county) == 'Laporte') {
        excludedbook = read_excel(paste0('tippecanoe/data/2024 Ratio Study - Sales Reconciliation Submissions/LaPorte County Sales Reconciliation.xlsx'))  
    } else if (gsub(" ", "", county) %in% c('Jasper','Pike')) {
        excludedbook = read_excel(paste0('tippecanoe/data/2024 Ratio Study - Sales Reconciliation Submissions/',gsub(" ", "", county),' County Reconciliation.xlsx'))  
    } else if (gsub(" ", "", county) %in% c('St.Joseph')) {
        excludedbook = read_excel(paste0('tippecanoe/data/2024 Ratio Study - Sales Reconciliation Submissions/St. Joseph County Sales Reconciliation.xlsx'))  
    } else if (gsub(" ", "", county) %in% c('Benton')) {
        excludedbook = read_excel(paste0('tippecanoe/data/2024 Ratio Study - Sales Reconciliation Submissions/Benton County Sales Reconciliation with notes.xlsx'))  
    } else if (gsub(" ", "", county) %in% c('Clark')) {
        excludedbook = read_excel(paste0('tippecanoe/data/2024 Ratio Study - Sales Reconciliation Submissions/Clark County Sales Reconciliation Excluded Sales.xlsx'))  
    } else if (gsub(" ", "", county) %in% c('Morgan')) {
        excludedbook = read_excel(paste0('tippecanoe/data/2024 Ratio Study - Sales Reconciliation Submissions/Morgan County Sales Reconciliation with explanations.xlsx'))  
    } else if (gsub(" ", "", county) %in% c('Hamilton')) {
        excludedbook = read_excel(paste0('tippecanoe/data/2024 Ratio Study - Sales Reconciliation Submissions/Hamilton County Sales Reconciliation - with explanations.xlsx'))  
    } else {
        excludedbook = read_excel(paste0('tippecanoe/data/2024 Ratio Study - Sales Reconciliation Submissions/',gsub(" ", "", county),' County Sales Reconciliation.xlsx'))
    }
    if (gsub(" ", "", county) %in% c('Scott')) {
        colnames(excludedbook)[colnames(excludedbook) == "FormattedParcelNumber"] <- "Formatted Parcel Number"
        colnames(excludedbook)[colnames(excludedbook) == "PropertyClass"] <- "Property Class"
        colnames(excludedbook)[colnames(excludedbook) == "TotalAV"] <- "Total AV"
        colnames(excludedbook)[colnames(excludedbook) == "SalePrice"] <- "Sale Price"
        colnames(excludedbook)[colnames(excludedbook) == "SaleRatio"] <- "Sale Ratio"
    }
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("included", Explanation, ignore.case = TRUE) ~ "ON RATIO STUDY",
        grepl("ratio study", Explanation, ignore.case = TRUE) ~ "ON RATIO STUDY",
        grepl("used", Explanation, ignore.case = TRUE) ~ "ON RATIO STUDY",
        grepl("in study", Explanation, ignore.case = TRUE) ~ "ON RATIO STUDY",
        grepl("validsale", Explanation, ignore.case = TRUE) ~ "ON RATIO STUDY",
        grepl("-valid sale", Explanation, ignore.case = TRUE) ~ "ON RATIO STUDY",
        grepl("formatted tab", Explanation, ignore.case = TRUE) ~ "ON RATIO STUDY",
        Explanation == "VALID" ~ "ON RATIO STUDY",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook = excludedbook[excludedbook$Explanation != "ON RATIO STUDY", ]
    postedexp <- ifelse(is.na(excludedbook$Explanation), "Blank", excludedbook$Explanation)
    if (gsub(" ", "", county) %in% c('Noble','Lagrange')) {
        excludedbook$Explanation[(is.na(excludedbook$Explanation))] = 'NO REASON PROVIDED'
    }
    if (!("Explanation" %in% colnames(excludedbook))) {
        message("Column 'Explanation' not found in ", county)
    }
    if (!("Formatted Parcel Number" %in% colnames(excludedbook))) {
        message("Column 'Formatted Parcel Number' not found in ", county)
    }
    if (!("Total AV" %in% colnames(excludedbook))) {
        message("Column 'Total AV' not found in ", county)
    }
    if (!("Sale Price" %in% colnames(excludedbook))) {
        message("Column 'Sale Price' not found in ", county)
    }
    if (!("Sale Ratio" %in% colnames(excludedbook))) {
        message("Column 'Sale Ratio' not found in ", county)
    }
    if (!("Property Class" %in% colnames(excludedbook))) {
        message("Column 'Property Class' not found in ", county)
    }
    
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("golf", Explanation, ignore.case = TRUE) ~ "GOLF COURSE",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("LEASE", Explanation, ignore.case = TRUE) ~ "LEASE",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("campground", Explanation, ignore.case = TRUE) ~ "CAMPGROUND",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("common", Explanation, ignore.case = TRUE) ~ "COMMON GROUND",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("permit", Explanation, ignore.case = TRUE) ~ "BUILDING PERMIT",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("MOBILE", Explanation, ignore.case = TRUE) ~ "MOBILE HOME",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("special use", Explanation, ignore.case = TRUE) ~ "SPECIAL USE",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("Adja", Explanation, ignore.case = TRUE) ~ "ADJACENT OWNER",
        grepl("Adjo", Explanation, ignore.case = TRUE) ~ "ADJACENT OWNER",
        grepl("next door", Explanation, ignore.case = TRUE) ~ "ADJACENT OWNER",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("duplicate", Explanation, ignore.case = TRUE) ~ "DUPLICATE",
        grepl("dup sumb", Explanation, ignore.case = TRUE) ~ "DUPLICATE",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("PARTIAL", Explanation, ignore.case = TRUE) ~ "PARTIAL INTEREST",
        grepl("Percent int", Explanation, ignore.case = TRUE) ~ "PARTIAL INTEREST",
        grepl("half int", Explanation, ignore.case = TRUE) ~ "PARTIAL INTEREST",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("INVEST", Explanation, ignore.case = TRUE) ~ "INVESTMENT/RENTAL",
        grepl("RENT", Explanation, ignore.case = TRUE) ~ "INVESTMENT/RENTAL",
        grepl("INCOME", Explanation, ignore.case = TRUE) ~ "INVESTMENT/RENTAL",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("sold as is", Explanation, ignore.case = TRUE) ~ "SOLD AS IS",
        grepl("as-is", Explanation, ignore.case = TRUE) ~ "SOLD AS IS",
        grepl("as is", Explanation, ignore.case = TRUE) ~ "SOLD AS IS",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("MIXED", Explanation, ignore.case = TRUE) ~ "RELATED ASSETS",
        grepl("PORTFOLIO", Explanation, ignore.case = TRUE) ~ "RELATED ASSETS",
        grepl("mix use", Explanation, ignore.case = TRUE) ~ "RELATED ASSETS",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("apt", Explanation, ignore.case = TRUE) ~ "APARTMENT",
        grepl("apartment", Explanation, ignore.case = TRUE) ~ "APARTMENT",
        grepl("condo", Explanation, ignore.case = TRUE) ~ "APARTMENT",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("OUTSIDE", Explanation, ignore.case = TRUE) ~ "OUTSIDE TIMEFRAME",
        grepl("time frame", Explanation, ignore.case = TRUE) ~ "OUTSIDE TIMEFRAME",
        grepl("future sale", Explanation, ignore.case = TRUE) ~ "OUTSIDE TIMEFRAME",
        grepl("conveyance date", Explanation, ignore.case = TRUE) ~ "OUTSIDE TIMEFRAME",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("CONSTRUCTION", Explanation, ignore.case = TRUE) ~ "NEW CONSTRUCTION",
        grepl("new plat", Explanation, ignore.case = TRUE) ~ "NEW CONSTRUCTION",
        grepl("new home", Explanation, ignore.case = TRUE) ~ "NEW CONSTRUCTION",
        grepl("new house", Explanation, ignore.case = TRUE) ~ "NEW CONSTRUCTION",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("prior", Explanation, ignore.case = TRUE) ~ "CHANGES PRIOR",
        grepl("before sale", Explanation, ignore.case = TRUE) ~ "CHANGES PRIOR",
        grepl("changes before", Explanation, ignore.case = TRUE) ~ "CHANGES PRIOR",
        grepl("changebefore", Explanation, ignore.case = TRUE) ~ "CHANGES PRIOR",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("CLASS", Explanation, ignore.case = TRUE) ~ "INVALID PROPERTY CLASS",
        grepl("commercial", Explanation, ignore.case = TRUE) ~ "INVALID PROPERTY CLASS",
        grepl("not com", Explanation, ignore.case = TRUE) ~ "INVALID PROPERTY CLASS",
        grepl("notcom", Explanation, ignore.case = TRUE) ~ "INVALID PROPERTY CLASS",
        grepl("type changed", Explanation, ignore.case = TRUE) ~ "INVALID PROPERTY CLASS",
        grepl("industrial", Explanation, ignore.case = TRUE) ~ "INVALID PROPERTY CLASS",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("LIMITED", Explanation, ignore.case = TRUE) ~ "LIMITED INFO",
        grepl("INFO", Explanation, ignore.case = TRUE) ~ "LIMITED INFO",
        grepl("NAME CHANGE", Explanation, ignore.case = TRUE) ~ "LIMITED INFO",
        grepl("No response", Explanation, ignore.case = TRUE) ~ "LIMITED INFO",
        grepl("unable to verify", Explanation, ignore.case = TRUE) ~ "LIMITED INFO",
        grepl("no sale price", Explanation, ignore.case = TRUE) ~ "LIMITED INFO",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("sufficient", Explanation, ignore.case = TRUE) ~ "INSUFFICIENT TIME ON MARKET",
        grepl("TIME ON MARKET", Explanation, ignore.case = TRUE) ~ "INSUFFICIENT TIME ON MARKET",
        grepl("TIME ON THE MARKET", Explanation, ignore.case = TRUE) ~ "INSUFFICIENT TIME ON MARKET",
        grepl("same day", Explanation, ignore.case = TRUE) ~ "INSUFFICIENT TIME ON MARKET",
        grepl("1 day", Explanation, ignore.case = TRUE) ~ "INSUFFICIENT TIME ON MARKET",
        grepl("one day", Explanation, ignore.case = TRUE) ~ "INSUFFICIENT TIME ON MARKET",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("trend", Explanation, ignore.case = TRUE) ~ "NOT ENOUGH SALES FOR TRENDING",
        grepl("not enough", Explanation, ignore.case = TRUE) ~ "NOT ENOUGH SALES FOR TRENDING",
        grepl("to study", Explanation, ignore.case = TRUE) ~ "NOT ENOUGH SALES FOR TRENDING",
        grepl("to do study", Explanation, ignore.case = TRUE) ~ "NOT ENOUGH SALES FOR TRENDING",
        grepl("lack of sales", Explanation, ignore.case = TRUE) ~ "NOT ENOUGH SALES FOR TRENDING",
        grepl("not using", Explanation, ignore.case = TRUE) ~ "NOT ENOUGH SALES FOR TRENDING",
        grepl("sales for study", Explanation, ignore.case = TRUE) ~ "NOT ENOUGH SALES FOR TRENDING",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("RELIGIO", Explanation, ignore.case = TRUE) ~ "SOLD TO EXEMPT ORGANIZATION",
        grepl("PROFIT", Explanation, ignore.case = TRUE) ~ "SOLD TO EXEMPT ORGANIZATION",
        grepl("GOV", Explanation, ignore.case = TRUE) ~ "SOLD TO EXEMPT ORGANIZATION",
        grepl("CHARIT", Explanation, ignore.case = TRUE) ~ "SOLD TO EXEMPT ORGANIZATION",
        grepl("EDUCATI", Explanation, ignore.case = TRUE) ~ "SOLD TO EXEMPT ORGANIZATION",
        grepl("school", Explanation, ignore.case = TRUE) ~ "SOLD TO EXEMPT ORGANIZATION",
        grepl("exempt", Explanation, ignore.case = TRUE) ~ "SOLD TO EXEMPT ORGANIZATION",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("sold for less than $", Explanation, ignore.case = TRUE) ~ "LOW SALE PRICE",
        grepl("\\$\\w+,*\\w+\\s*SALE", Explanation, ignore.case = TRUE) ~ "LOW SALE PRICE",
        grepl("low sale price", Explanation, ignore.case = TRUE) ~ "LOW SALE PRICE",
        grepl("ZERO sales price", Explanation, ignore.case = TRUE) ~ "LOW SALE PRICE",
        grepl("sale amount less", Explanation, ignore.case = TRUE) ~ "LOW SALE PRICE",
        grepl("sale less th", Explanation, ignore.case = TRUE) ~ "LOW SALE PRICE",
        grepl("sold for under", Explanation, ignore.case = TRUE) ~ "LOW SALE PRICE",
        grepl("under $", Explanation, ignore.case = TRUE) ~ "LOW SALE PRICE",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("in use", Explanation, ignore.case = TRUE) ~ "CHANGE IN USE",
        grepl("PRIMARY USE", Explanation, ignore.case = TRUE) ~ "CHANGE IN USE",
        grepl("USE CHANGE", Explanation, ignore.case = TRUE) ~ "CHANGE IN USE",
        grepl("CHANGE IN", Explanation, ignore.case = TRUE) ~ "CHANGE IN USE",
        grepl("CHANGE UN", Explanation, ignore.case = TRUE) ~ "CHANGE IN USE",
        grepl("changed use", Explanation, ignore.case = TRUE) ~ "CHANGE IN USE",
        grepl("change of use", Explanation, ignore.case = TRUE) ~ "CHANGE IN USE",
        grepl("changeinuse", Explanation, ignore.case = TRUE) ~ "CHANGE IN USE",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("vac", Explanation, ignore.case = TRUE) ~ "VACANT LAND",
        grepl("v/L", Explanation, ignore.case = TRUE) ~ "VACANT LAND",
        grepl("INACTIVE", Explanation, ignore.case = TRUE) ~ "VACANT LAND",
        grepl("abandoned", Explanation, ignore.case = TRUE) ~ "VACANT LAND",
        grepl("bare", Explanation, ignore.case = TRUE) ~ "VACANT LAND",
        Explanation == "VA" ~ "VACANT LAND",
        grepl("HOMESITE", Explanation, ignore.case = TRUE) ~ "VACANT LAND",
        grepl("dwelling", Explanation, ignore.case = TRUE) ~ "VACANT LAND",
        grepl("residential structure", Explanation, ignore.case = TRUE) ~ "VACANT LAND",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("dev", Explanation, ignore.case = TRUE) ~ "BUSINESS TRANSACTION",
        grepl("BUILDER", Explanation, ignore.case = TRUE) ~ "BUSINESS TRANSACTION",
        grepl("BUSINESS", Explanation, ignore.case = TRUE) ~ "BUSINESS TRANSACTION",
        grepl("CONSOLID", Explanation, ignore.case = TRUE) ~ "BUSINESS TRANSACTION",
        grepl("REBUILD", Explanation, ignore.case = TRUE) ~ "BUSINESS TRANSACTION",
        grepl("llc", Explanation, ignore.case = TRUE) ~ "BUSINESS TRANSACTION",
        grepl("holdings company", Explanation, ignore.case = TRUE) ~ "BUSINESS TRANSACTION",
        grepl("lending inst", Explanation, ignore.case = TRUE) ~ "BUSINESS TRANSACTION",
        grepl("reo", Explanation, ignore.case = TRUE) ~ "BUSINESS TRANSACTION",
        grepl("corporat", Explanation, ignore.case = TRUE) ~ "BUSINESS TRANSACTION",
        grepl("MERGER", Explanation, ignore.case = TRUE) ~ "BUSINESS TRANSACTION",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("not exist", Explanation, ignore.case = TRUE) ~ "INVALID PARCEL NUMBER",
        grepl("ACTIVE PARCEL", Explanation, ignore.case = TRUE) ~ "INVALID PARCEL NUMBER",
        grepl("wrong parcel", Explanation, ignore.case = TRUE) ~ "INVALID PARCEL NUMBER",
        grepl("no longer a parcel", Explanation, ignore.case = TRUE) ~ "INVALID PARCEL NUMBER",
        grepl("no longer exists", Explanation, ignore.case = TRUE) ~ "INVALID PARCEL NUMBER",
        grepl("retired", Explanation, ignore.case = TRUE) ~ "INVALID PARCEL NUMBER",
        grepl("is not active", Explanation, ignore.case = TRUE) ~ "INVALID PARCEL NUMBER",
        grepl("not an active", Explanation, ignore.case = TRUE) ~ "INVALID PARCEL NUMBER",
        grepl("deleted parcel", Explanation, ignore.case = TRUE) ~ "INVALID PARCEL NUMBER",
        grepl("parcelisnolonger", Explanation, ignore.case = TRUE) ~ "INVALID PARCEL NUMBER",
        grepl("no longer active", Explanation, ignore.case = TRUE) ~ "INVALID PARCEL NUMBER",
        grepl("no loner", Explanation, ignore.case = TRUE) ~ "INVALID PARCEL NUMBER",
        TRUE ~ Explanation # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("BANKRUP", Explanation, ignore.case = TRUE) ~ "FORCED SALE",
        grepl("FORECLO", Explanation, ignore.case = TRUE) ~ "FORCED SALE",
        grepl("DIVORCE", Explanation, ignore.case = TRUE) ~ "FORCED SALE",
        grepl("forced", Explanation, ignore.case = TRUE) ~ "FORCED SALE",
        grepl("distressed", Explanation, ignore.case = TRUE) ~ "FORCED SALE",
        grepl("duress", Explanation, ignore.case = TRUE) ~ "FORCED SALE",
        grepl("hardship", Explanation, ignore.case = TRUE) ~ "FORCED SALE",
        grepl("court", Explanation, ignore.case = TRUE) ~ "FORCED SALE",
        grepl("death", Explanation, ignore.case = TRUE) ~ "FORCED SALE",
        grepl("compulsory", Explanation, ignore.case = TRUE) ~ "FORCED SALE",
        grepl("repo", Explanation, ignore.case = TRUE) ~ "FORCED SALE",
        grepl("judicial", Explanation, ignore.case = TRUE) ~ "FORCED SALE",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("AFTER SALE", Explanation, ignore.case = TRUE) ~ "CHANGES AFTER",
        grepl("SIGNIFICANT", Explanation, ignore.case = TRUE) ~ "CHANGES AFTER",
        grepl("PHYSICAL CHANGE", Explanation, ignore.case = TRUE) ~ "CHANGES AFTER",
        grepl("remodel", Explanation, ignore.case = TRUE) ~ "CHANGES AFTER",
        grepl("reno", Explanation, ignore.case = TRUE) ~ "CHANGES AFTER",
        grepl("changes after", Explanation, ignore.case = TRUE) ~ "CHANGES AFTER",
        grepl("improve", Explanation, ignore.case = TRUE) ~ "CHANGES AFTER",
        grepl("add", Explanation, ignore.case = TRUE) ~ "CHANGES AFTER",
        grepl("updat", Explanation, ignore.case = TRUE) ~ "CHANGES AFTER",
        grepl("post sale", Explanation, ignore.case = TRUE) ~ "CHANGES AFTER",
        grepl("since last assessment", Explanation, ignore.case = TRUE) ~ "CHANGES AFTER",
        grepl("after the sale", Explanation, ignore.case = TRUE) ~ "CHANGES AFTER",
        grepl("changeafter", Explanation, ignore.case = TRUE) ~ "CHANGES AFTER",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("twice", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        grepl("2X", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        grepl("more than", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        grepl("more then", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        grepl("three times", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        grepl("resold", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        grepl("re-sold", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        grepl("2 sales", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        grepl("first of two", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        grepl("2 times", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        grepl("4 times", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        grepl("3 times", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        grepl("second sale", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        grepl("also sold", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        grepl("again", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        grepl("later date", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        grepl("same parcel sold", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        grepl("months later", Explanation, ignore.case = TRUE) ~ "SOLD MORE THAN ONCE IN ONE YEAR",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("family", Explanation, ignore.case = TRUE) ~ "PRE-EXISTING RELATIONSHIP",
        grepl("relat", Explanation, ignore.case = TRUE) ~ "PRE-EXISTING RELATIONSHIP",
        grepl("LATive", Explanation, ignore.case = TRUE) ~ "PRE-EXISTING RELATIONSHIP",
        grepl("AFFILIA", Explanation, ignore.case = TRUE) ~ "PRE-EXISTING RELATIONSHIP",
        grepl("SAME OWNER", Explanation, ignore.case = TRUE) ~ "PRE-EXISTING RELATIONSHIP",
        grepl("S LENGTH", Explanation, ignore.case = TRUE) ~ "PRE-EXISTING RELATIONSHIP",
        grepl("IS TENANT", Explanation, ignore.case = TRUE) ~ "PRE-EXISTING RELATIONSHIP",
        grepl("same company", Explanation, ignore.case = TRUE) ~ "PRE-EXISTING RELATIONSHIP",
        grepl("inherit", Explanation, ignore.case = TRUE) ~ "PRE-EXISTING RELATIONSHIP",
        grepl("same person", Explanation, ignore.case = TRUE) ~ "PRE-EXISTING RELATIONSHIP",
        grepl("same entity", Explanation, ignore.case = TRUE) ~ "PRE-EXISTING RELATIONSHIP",
        grepl("friend", Explanation, ignore.case = TRUE) ~ "PRE-EXISTING RELATIONSHIP",
        grepl("arms len", Explanation, ignore.case = TRUE) ~ "PRE-EXISTING RELATIONSHIP",
        grepl("arms-length", Explanation, ignore.case = TRUE) ~ "PRE-EXISTING RELATIONSHIP",
        grepl("ated org", Explanation, ignore.case = TRUE) ~ "PRE-EXISTING RELATIONSHIP",
        grepl("survivorship", Explanation, ignore.case = TRUE) ~ "PRE-EXISTING RELATIONSHIP",
        grepl("probate", Explanation, ignore.case = TRUE) ~ "PRE-EXISTING RELATIONSHIP",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("AUCTION", Explanation, ignore.case = TRUE) ~ "UNIQUE SALE",
        grepl("ESTATE", Explanation, ignore.case = TRUE) ~ "UNIQUE SALE",
        grepl("TRUST", Explanation, ignore.case = TRUE) ~ "UNIQUE SALE",
        grepl("BANK", Explanation, ignore.case = TRUE) ~ "UNIQUE SALE",
        grepl("CASH", Explanation, ignore.case = TRUE) ~ "UNIQUE SALE",
        grepl("QUIT", Explanation, ignore.case = TRUE) ~ "UNIQUE SALE",
        grepl("QC", Explanation, ignore.case = TRUE) ~ "UNIQUE SALE",
        grepl("TAX", Explanation, ignore.case = TRUE) ~ "UNIQUE SALE",
        grepl("SHORT SALE", Explanation, ignore.case = TRUE) ~ "UNIQUE SALE",
        grepl("CONTRACT", Explanation, ignore.case = TRUE) ~ "UNIQUE SALE",
        grepl("HUD", Explanation, ignore.case = TRUE) ~ "UNIQUE SALE",
        grepl("GUARDIANSHIP", Explanation, ignore.case = TRUE) ~ "UNIQUE SALE",
        grepl("sheriff", Explanation, ignore.case = TRUE) ~ "UNIQUE SALE",
        grepl("lc payoff", Explanation, ignore.case = TRUE) ~ "UNIQUE SALE",
        grepl("financ", Explanation, ignore.case = TRUE) ~ "UNIQUE SALE",
        grepl("trade", Explanation, ignore.case = TRUE) ~ "UNIQUE SALE",
        grepl("arbitration", Explanation, ignore.case = TRUE) ~ "UNIQUE SALE",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("FIXER", Explanation, ignore.case = TRUE) ~ "POOR CONDITION",
        grepl("flip", Explanation, ignore.case = TRUE) ~ "POOR CONDITION",
        grepl("CONDITION", Explanation, ignore.case = TRUE) ~ "POOR CONDITION",
        grepl("liveable", Explanation, ignore.case = TRUE) ~ "POOR CONDITION",
        grepl("uninhabit", Explanation, ignore.case = TRUE) ~ "POOR CONDITION",
        grepl("rehab", Explanation, ignore.case = TRUE) ~ "POOR CONDITION",
        grepl("fire", Explanation, ignore.case = TRUE) ~ "POOR CONDITION",
        grepl("remov", Explanation, ignore.case = TRUE) ~ "POOR CONDITION",
        grepl("bad shape", Explanation, ignore.case = TRUE) ~ "POOR CONDITION",
        grepl("demo", Explanation, ignore.case = TRUE) ~ "POOR CONDITION",
        grepl("needs rep", Explanation, ignore.case = TRUE) ~ "POOR CONDITION",
        grepl("disrepair", Explanation, ignore.case = TRUE) ~ "POOR CONDITION",
        grepl("contamination", Explanation, ignore.case = TRUE) ~ "POOR CONDITION",
        grepl("condemn", Explanation, ignore.case = TRUE) ~ "POOR CONDITION",
        grepl("rough shape", Explanation, ignore.case = TRUE) ~ "POOR CONDITION",
        grepl("burnt", Explanation, ignore.case = TRUE) ~ "POOR CONDITION",
        grepl("need of repair", Explanation, ignore.case = TRUE) ~ "POOR CONDITION",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("mul", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("MPS", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("parcel sale", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("package sale", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("combin", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("with another", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("split", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("2 properties", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("partition", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("2 parcel", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("contiguous", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("two parcel", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("2 part", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("two separate", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("group of", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("prices were arbit", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("4 parcels", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("bulk property", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("shares sale price", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("sold with", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        grepl("several", Explanation, ignore.case = TRUE) ~ "MULTI PARCEL SALE",
        TRUE ~ Explanation
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("open market", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("closed sale", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("hit the market", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("never hit", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("never listed", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("NOT listed", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("NOT on", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("fsbo", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("buyer approached", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("private", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("never on market", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("for sale by owner", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("personal rep", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("word of mouth", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("sold by owner", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("market time", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("expos", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("advertise", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("off market", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("not marketed", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("out of market", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("no listing", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("non marketed", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        grepl("never on the market", Explanation, ignore.case = TRUE) ~ "NOT LISTED ON OPEN MARKET",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
     excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        grepl("assessed", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("less than", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("discount", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("motivated", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("priced to sell", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("concessions", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("price drop", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("typical", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("below market", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("indicative", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("considerably", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("outlier", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("market variab", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("reflect", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("reduced", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("under market", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("neighborhood", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("represent", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("overpaid", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("intangible", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("per. prop", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("nontangible", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("sell off", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("goodwill", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("personal prop", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("person. prop", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("seller paid", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("appraisal completed", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("value set on appeal", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("asking price", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("recent sale", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("INCORRECT SALE PRICE", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("over av", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("errors in assessment", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("statistical range", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("unusual for market", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("yard items", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("knowledge of sale", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("belowmarket", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("undermarket", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("per.prop", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("furnish", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("bid", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("excessive price", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("PP ", Explanation, ignore.case = FALSE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("includes pp", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("% AV", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("% of AV", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("max depreciation", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("out building", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("outbuilding", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        grepl("UNKNOWLEDGEABLE", Explanation, ignore.case = TRUE) ~ "DOES NOT REFLECT MARKET VALUE",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
        mutate(Explanation = case_when(
        grepl("AG", Explanation, ignore.case = TRUE) ~ "AG LAND",
        grepl("farm", Explanation, ignore.case = TRUE) ~ "AG LAND",
        grepl("barn", Explanation, ignore.case = TRUE) ~ "AG LAND",
        TRUE ~ Explanation  # Keep other descriptions unchanged
      ))
    excludedbook <- excludedbook %>%
      mutate(Explanation = case_when(
        Explanation == "INVALID" ~ "NO REASON PROVIDED",
        Explanation == "Invalid" ~ "NO REASON PROVIDED",
        Explanation == "invalid" ~ "NO REASON PROVIDED",
        Explanation == "NOT VALID" ~ "NO REASON PROVIDED",
        Explanation == "not valid" ~ "NO REASON PROVIDED",
        Explanation == "INVALID-" ~ "NO REASON PROVIDED",
        Explanation == "invalid-" ~ "NO REASON PROVIDED",
        Explanation == "N/A" ~ "NO REASON PROVIDED",
        Explanation == " " ~ "NO REASON PROVIDED",
        grepl("other", Explanation, ignore.case = TRUE) ~ "NO REASON PROVIDED",
        grepl("do not use", Explanation, ignore.case = TRUE) ~ "NO REASON PROVIDED",
        TRUE ~ Explanation
      ))
        excluded_indices <- match(excludedbook$'Formatted Parcel Number',ctywkbk$ParcelNumber)
        postedAV = as.numeric(excludedbook$'Total AV')
        excludedbook$'Total AV'[!is.na(excluded_indices)] <- ctywkbk$CurrentAV[excluded_indices[!is.na(excluded_indices)]]
        newdata = data.frame(
          County = rep(county,nrow(excludedbook)),
          ParcelNumber = excludedbook$'Formatted Parcel Number',
          PropertyClass = excludedbook$'Property Class',
          PostedAV = postedAV, 
          CurrentAV = as.numeric(excludedbook$'Total AV'),
          SalePrice = as.numeric(excludedbook$'Sale Price'),
          PostedSaleRatio = postedAV / as.numeric(excludedbook$'Sale Price'),
          SaleRatio = as.numeric(excludedbook$'Total AV') / as.numeric(excludedbook$'Sale Price'),
          PostedExplanation = postedexp,
          Explanation = excludedbook$Explanation,
          Reclassified = !(postedexp == excludedbook$Explanation)
        )
        combinedexc24 = rbind(combinedexc24,newdata)
}
head(combinedexc24)

New names:
* `` -> `...13`
New names:
* `` -> `...13`
* `` -> `...14`
New names:
* `SDFID` -> `SDFID...2`
* `` -> `...13`
* `SDFID` -> `SDFID...14`


,County,ParcelNumber,PropertyClass,PostedAV,CurrentAV,SalePrice,PostedSaleRatio,SaleRatio,PostedExplanation,Explanation,Reclassified
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<lgl>
1,Adams,01-05-10-203-057.000-022,510,63100,78300,120000,0.525833333,0.6525000,Private Sale,NOT LISTED ON OPEN MARKET,TRUE
2,Adams,01-05-01-102-014.089-022,500,300,237900,45000,0.006666667,5.2866667,Significant Physical Change,CHANGES AFTER,TRUE
3,Adams,01-05-03-400-025.000-022,599,14400,18100,10000,1.440000000,1.8100000,Not typical Improvement for the Neighborhood,CHANGES AFTER,TRUE
4,Adams,01-02-35-402-026.000-014,510,84700,105300,142500,0.594385965,0.7389474,Private Sale,NOT LISTED ON OPEN MARKET,TRUE
5,Adams,01-05-03-113-087.000-022,510,99800,123300,68000,1.467647059,1.8132353,Sold Twice,SOLD MORE THAN ONCE IN ONE YEAR,TRUE
6,Adams,01-02-35-402-028.000-014,510,74800,85600,120000,0.623333333,0.7133333,Not Listed on Market,NOT LISTED ON OPEN MARKET,TRUE


In [16]:
combinedrat24 = data.frame(
  County = character(),
  ParcelNumber = character(),
  PropertyClass = integer(), 
  PostedAV = numeric(),
  CurrentAV = numeric(),
  SalePrice = numeric(),
  PostedSaleRatio = numeric(),
  SaleRatio = numeric()
)
for (county in names(myfiles24)) {
    ctywkbk = combinedwkbk24[combinedwkbk24$County == county,]
    if (gsub(" ", "", county) %in% c('Crawford','Spencer')) {
        next
        ## Crawford 2024 Approved Ratio Study is in .xls format
        ## Spencer 2024 Approved Ratio Study is in .xls format
    }
    
    filepath = paste0('tippecanoe/data/2024 Approved Ratio Studies/',toupper(county),' 2024 RATIO STUDY.xlsx')
    sheets <- excel_sheets(filepath)
    tryCatch({
        if (gsub(" ", "", county) %in% c('Ohio')) {
            ratiobook <- read_excel(filepath, sheet='Formatted ')
        } else {
            ratiobook <- read_excel(filepath, sheet = 'Formatted')
        }
    }, error = function(e) {
        message("Error reading the Excel file: ",county,e$message)
        # Optionally print the structure of the file or a sample
        message("Attempting to read the first few rows.")
        test_read <- read_excel(file_path, sheet = sheets[1])  # Read the first sheet
        print(head(test_read))  # Print the first few rows
    })
    if (gsub(" ", "", county) %in% c('Daviess','Laporte','Marshall','Monroe','Pike','Putnam','Ripley','Switzerland','Vanderburgh','Wabash','White')) {
        colnames(ratiobook)[colnames(ratiobook) == "ParcelNumber"] <- "Parcel Number"
        colnames(ratiobook)[colnames(ratiobook) == "PropertyClass"] <- "Property Class"
        colnames(ratiobook)[colnames(ratiobook) == "CurrentTotalAV"] <- "Current Total AV"
        colnames(ratiobook)[colnames(ratiobook) == "StudySalePrice"] <- "Study Sale Price"
    }
    if (gsub(" ", "", county) %in% c('Grant','Noble')) {
        colnames(ratiobook)[colnames(ratiobook) == "State Parcel Number"] <- "Parcel Number"
    }
    if (gsub(" ", "", county) %in% c('Randolph')) {
        colnames(ratiobook)[colnames(ratiobook) == "ParcelNumber"] <- "Parcel Number"
    }
    if (gsub(" ", "", county) %in% c('Steuben')) {
        colnames(ratiobook)[colnames(ratiobook) == "Propery Class"] <- "Property Class"
    }
    if (gsub(" ", "", county) %in% c('Lagrange')) {
        colnames(ratiobook)[colnames(ratiobook) == "Study Sales Price"] <- "Study Sale Price"
    }
    if (!("Parcel Number" %in% colnames(ratiobook))) {
        message("Column 'Parcel Number' not found in ", county)
    }
    if (!("Property Class" %in% colnames(ratiobook))) {
        message("Column 'Property Class' not found in ", county)
    }
    if (!("Current Total AV" %in% colnames(ratiobook))) {
        message("Column 'Current Total AV' not found in ", county)
    }
    if (!("Study Sale Price" %in% colnames(ratiobook))) {
        message("Column 'Study Sale Price' not found in ", county)
    }
    ratio_indices <- match(ratiobook$"Parcel Number",ctywkbk$ParcelNumber)
    postedAV = as.numeric(ratiobook$'Current Total AV')
    ratiobook$'Current Total AV'[!is.na(ratio_indices)] <- ctywkbk$CurrentAV[ratio_indices[!is.na(ratio_indices)]]
    newdata = data.frame(
      County = rep(county,nrow(ratiobook)),
      ParcelNumber = ratiobook$'Parcel Number',
      PropertyClass = ratiobook$'Property Class', 
      PostedAV = postedAV,
      CurrentAV = as.numeric(ratiobook$'Current Total AV'),
      SalePrice = as.numeric(ratiobook$'Study Sale Price'),
      PostedSaleRatio = postedAV / as.numeric(ratiobook$'Study Sale Price'),
      SaleRatio = as.numeric(ratiobook$'Current Total AV') / as.numeric(ratiobook$'Study Sale Price')
    )
    combinedrat24 = rbind(combinedrat24,newdata)
}
head(combinedrat24)

New names:
* `` -> `...16`
* `` -> `...17`
* `` -> `...18`
* `` -> `...19`
* `` -> `...20`
* `` -> `...21`
* `` -> `...22`
* `` -> `...23`
* `` -> `...24`
* `` -> `...25`
* `` -> `...26`
* `` -> `...27`
* `` -> `...28`
* `` -> `...29`
* `` -> `...30`
* `` -> `...31`
* `` -> `...32`
* `` -> `...33`
* `` -> `...34`
* `` -> `...35`
* `` -> `...36`
* `` -> `...37`
* `` -> `...38`
* `` -> `...39`
* `` -> `...40`
* `` -> `...41`
* `` -> `...42`
* `` -> `...43`
* `` -> `...44`
* `` -> `...45`
* `` -> `...46`
* `` -> `...47`
* `` -> `...48`
* `` -> `...49`
* `` -> `...50`
* `` -> `...51`
* `` -> `...52`
* `` -> `...53`
* `` -> `...54`
* `` -> `...55`
* `` -> `...56`
* `` -> `...57`
* `` -> `...58`
* `` -> `...59`
* `` -> `...60`
* `` -> `...61`
* `` -> `...62`
* `` -> `...63`
* `` -> `...64`
* `` -> `...65`
* `` -> `...66`
* `` -> `...67`
* `` -> `...68`
* `` -> `...69`
* `` -> `...70`
* `` -> `...71`
* `` -> `...72`
* `` -> `...73`
* `` -> `...74`
* `` -> `...75`
* `` -> `...76`
* `` -> `...7

,County,ParcelNumber,PropertyClass,PostedAV,CurrentAV,SalePrice,PostedSaleRatio,SaleRatio
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Adams,01-02-25-100-002.000-013,511,162400,162400,154800,1.0490956,1.0490956
2,Adams,01-02-21-400-009.000-013,511,243600,243600,262200,0.9290618,0.9290618
3,Adams,01-05-04-204-018.000-022,510,136900,136900,104500,1.3100478,1.3100478
4,Adams,01-05-02-101-028.000-022,510,181900,181900,172700,1.0532716,1.0532716
5,Adams,01-02-36-301-022.000-014,510,263400,263400,268000,0.9828358,0.9828358
6,Adams,01-05-03-115-016.000-022,510,101700,101700,101600,1.0009843,1.0009843


In [34]:
nrow(combinedexc23[!(combinedexc23$Reclassified),]) / nrow(combinedexc23) * 100
nrow(combinedexc24[!(combinedexc24$Reclassified),]) / nrow(combinedexc24) * 100

[1] 1.858482

[1] 2.313906

In [65]:
head(combinedexc23[combinedexc23$Explanation == 'SOLD AS IS',],10)

,County,ParcelNumber,PropertyClass,PostedAV,CurrentAV,SalePrice,PostedSaleRatio,SaleRatio,PostedExplanation,Explanation,Reclassified
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<lgl>
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA.1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4612,Blackford,05-03-10-401-112.000-006,510,49600,51800,23000,2.1565217,2.2521739,House needs work sold as is,SOLD AS IS,TRUE
4923,Boone,06-08-04-000-008.000-016,511,485700,462500,315000,1.5419048,1.4682540,"Sold lower than market, seller unwilling to fix listed problems (as-is).",SOLD AS IS,TRUE
5064,Boone,06-11-30-000-003.027-002,510,303700,303700,257500,1.1794175,1.1794175,"sold under market. seller was unwilling to make repairs, ""as-is""",SOLD AS IS,TRUE
5068,Boone,06-13-35-000-024.000-015,511,173700,114700,105000,1.6542857,1.0923810,INVALID - SOLD AS IS,SOLD AS IS,TRUE
10972,Clinton,12-06-24-251-007.000-018,510,188800,207000,290000,0.6510345,0.7137931,SOLD AS IS,SOLD AS IS,FALSE
10999,Clinton,12-09-14-300-003.000-020,511,122300,122300,195000,0.6271795,0.6271795,SOLD AS IS,SOLD AS IS,FALSE
11173,Clinton,12-10-11-251-008.000-021,510,72300,81200,118000,0.6127119,0.6881356,SOLD AS IS,SOLD AS IS,FALSE


In [66]:
write.csv(combinedwkbk23, 'tippecanoe/Combined_Workbook_2023.csv',row.names=FALSE)
write.csv(combinedwkbk24, 'tippecanoe/Combined_Workbook_2024.csv',row.names=FALSE)
write.csv(combinedexc23, 'tippecanoe/Combined_Reconciliation_2023.csv',row.names=FALSE)
write.csv(combinedexc24, 'tippecanoe/Combined_Reconciliation_2024.csv',row.names=FALSE)
write.csv(combinedrat23, 'tippecanoe/Combined_Study_2023.csv',row.names=FALSE)
write.csv(combinedrat24, 'tippecanoe/Combined_Study_2024.csv',row.names=FALSE)